# h2o benchmarks - query 7

In [2]:
import coiled
import dask
import dask.dataframe as dd

In [5]:
coiled.list_instance_types(backend='aws')

{'t3a.medium': {'name': 't3a.medium',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 4096,
  'backend_type': 'vm_aws'},
 't3.medium': {'name': 't3.medium',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 4096,
  'backend_type': 'vm_aws'},
 't2.medium': {'name': 't2.medium',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 4096,
  'backend_type': 'vm_aws'},
 'm3.medium': {'name': 'm3.medium',
  'cores': 1,
  'gpus': 0,
  'gpu_name': None,
  'memory': 3840,
  'backend_type': 'vm_aws'},
 't3a.large': {'name': 't3a.large',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 8192,
  'backend_type': 'vm_aws'},
 'c6a.large': {'name': 'c6a.large',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 4096,
  'backend_type': 'vm_aws'},
 'c5a.large': {'name': 'c5a.large',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memory': 4096,
  'backend_type': 'vm_aws'},
 't3.large': {'name': 't3.large',
  'cores': 2,
  'gpus': 0,
  'gpu_name': None,
  'memo

In [7]:
cluster = coiled.Cluster(
    name="powers-crt-004",
    software="crt-004",
    n_workers=16,
    worker_vm_types=["i3.large"],
)

Output()

In [8]:
client = dask.distributed.Client(cluster)

## 5 GB dataset

In [9]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v2"],
)

In [10]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 627 ms, sys: 196 ms, total: 822 ms
Wall time: 44.7 s


,range_v1_v2
id3,
id0000608844,4
id0000466449,4
id0000573987,4
id0000776204,4
id0000608718,4
...,...
id0000821599,4
id0000937302,4
id0000248458,4


In [11]:
len(ddf)

100000000

In [12]:
ddf.npartitions

100

In [13]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg(
    {"v1": "max", "v2": "min"}, split_out=2
).assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]].compute()

CPU times: user 1.44 s, sys: 245 ms, total: 1.68 s
Wall time: 56.8 s


,range_v1_v2
id3,
id0000573987,4
id0000776204,4
id0000545596,4
id0000339296,4
id0000226214,4
...,...
id0000710890,4
id0000549495,4
id0000937302,4


In [14]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg(
    {"v1": "max", "v2": "min"}, split_out=20
).assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]].compute()

CPU times: user 13.3 s, sys: 1.94 s, total: 15.2 s
Wall time: 3min 42s


,range_v1_v2
id3,
id0000280302,4
id0000285456,4
id0000276808,4
id0000997868,4
id0000476252,4
...,...
id0000236113,4
id0000867213,4
id0000801592,4


## 50 GB dataset

In [15]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v2"],
)

In [16]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

KeyboardInterrupt: 

In [17]:
len(ddf)

1000000000

In [18]:
ddf.npartitions

2000

In [ ]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg(
    {"v1": "max", "v2": "min"}, split_out=10
).assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]].compute()